In [1]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project_client = AIProjectClient(
        endpoint=project_connection_string,
        credential=DefaultAzureCredential()
    )

except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise


In [2]:
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

connection_string = project_client.telemetry.get_application_insights_connection_string()
print(connection_string)
configure_azure_monitor(connection_string=connection_string)
OpenAIInstrumentor().instrument()


InstrumentationKey=d2ce280a-ce18-46fc-84c9-6915f61bd68f;IngestionEndpoint=https://southcentralus-3.in.applicationinsights.azure.com/;LiveEndpoint=https://southcentralus.livediagnostics.monitor.azure.com/;ApplicationId=dabc5447-e048-46a0-9125-f2c9f2f5631b


Statsbeat re-initialized with the same configuration.


In [3]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter

span_exporter = ConsoleSpanExporter()
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter))
trace.set_tracer_provider(tracer_provider)

In [4]:
client = project_client.get_openai_client()
response = client.chat.completions.create(
     model="gpt-4.1", 
     messages=[{"role": "user", "content": "Write a short poem on open telemetry."}],
)
print(response.choices[0].message.content)

c:\sandbox\agents-getting-started\newvenv\Lib\site-packages\opentelemetry\sdk\_events\__init__.py:53: LogDeprecatedInitWarning: LogRecord init with `trace_id`, `span_id`, and/or `trace_flags` is deprecated since 1.35.0. Use `context` instead.
  log_record = LogRecord(


In a world of data, vast and wide,  
OpenTelemetry is our guide.  
Traces, metrics, logs in tow,  
Following journeys systems go.

Unifying signals, breaking walls,  
Insight in the bytes it hauls.  
From app to server, cloud and thread,  
It maps the path our packets tread.

Open, free—community’s art,  
It helps observability start.  
With clarity shining, faults set free,  
We debug our dreams with OTel’s key.


In [5]:
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

current_span = trace.get_current_span()

current_span.set_attribute("operation.claims_count", "testing")
